# Semantically chunking based on sliding token windows and similarity reversions (Sliding Tokens Similarity Reversion)

This work is inteded to be an improvement on to of the semantic chunking work presented by [Greg Kamradt](https://twitter.com/GregKamradt), [previous work here.](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

The goal of this algorithm is to have a dynamic way of distinguishing when chunks of text change semantic meaning. The algorithm must not use tunable parameters, be dynamic and not rely on Agentic Chunking [Greg Kamradts work explains agentic chunking as well](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb).

The algorithm currently, creatively named "Sliding Tokens Similarity Reversion" achieves the goal, with good performance.

The steps are rather simple:
1. Tokenize an input text into a list of sentences
2. Create a sliding window over this list of sentences, merging two sentences together (the last sentence of one window will be the beginning of the next window)
3. Embed the new sliding tokens
4. Calculate the cosine similarity between the tokens
5. Using the cosine similarity, find the local minimum similarities and chunk based on those points
6. Steps above can be repeated to increase chunk size

Below is a figure showing the different steps. 
1. a0, b1, c2, are the naming for a sentence; a0 is the first sentence, b1 is the second sentence, c2 is the third sentence etc
2. a0b1, b1c2 etc are the sliding token windows
3. e0, e1, e2 are the embeddings of the sliding token windows
3. S0, S1, S2 etc are the similarities calculated between the embeddings -> e0 * e1 = s0

![Flow diagram](img/flow-diagram.png)


The algorithm was created by [Nesbyte](https://github.com/nesbyte). Attributions are always welcome. Please feel free to provide feedback on improvements and/or usecases. MIT License.


## Implementation and demonstration
*NOTE: This notebook implementation relies on [Ollama](https://ollama.com/) and uses [nomic-embed-text](https://ollama.com/library/nomic-embed-text) to perform the embeddings.*

In [1]:
import requests as rq
import nltk
nltk.download("punkt")

import json
from sklearn.metrics import pairwise
import numpy as np

from colorama import Fore

[nltk_data] Downloading package punkt to /home/stein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Helper funtion used to print the results in a more colourful output
def print_chunks_coloured(chunks: list[str]) -> str:
    colour_flag = True

    for chunk in chunks:
        if colour_flag:
            colour = Fore.BLUE
        else:
            colour = Fore.GREEN

        colour_flag = not colour_flag

        print(f"{colour}{chunk}{Fore.RESET}")
    

A piece from [Wikipedia Brazil page](https://en.wikipedia.org/wiki/Brazil)
Informational bodies of text is arguably the most interesting types of content to handle well. The text has been put into one big block on purpose to make the algorithm have full control over finding the semantic chunks. **More examples of different styles of text are shown at the end of the notebook**

In [3]:
input_text = """Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District. It is the only country in the Americas where Portuguese is an official language. Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world. Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). Covering roughly half of South America's land area, it borders all other countries and territories on the continent except Ecuador and Chile. Brazil's Amazon basin includes a vast tropical forest home to diverse wildlife, a variety of ecological systems, and extensive natural resources spanning numerous protected habitats. This unique environmental heritage positions Brazil at number one of 17 megadiverse countries. The country's natural richness is also the subject of significant global interest, as environmental degradation (through processes such as deforestation) has direct impacts on global issues such as climate change and biodiversity loss. The territory of present-day Brazil was inhabited by numerous tribal nations prior to the landing of explorer Pedro Álvares Cabral in 1500. Subsequently claimed by the Portuguese Empire, Brazil remained a Portuguese colony until 1808, when the capital of the empire was transferred from Lisbon to Rio de Janeiro. In 1815, the colony was elevated to the rank of kingdom upon the formation of the United Kingdom of Portugal, Brazil and the Algarves. Independence was achieved in 1822 with the creation of the Empire of Brazil, a unitary state governed under a constitutional monarchy and a parliamentary system. The ratification of the first constitution in 1824 led to the formation of a bicameral legislature, now called the National Congress. Slavery was abolished in 1888. The country became a presidential republic in 1889 following a military coup d'état. An authoritarian military dictatorship emerged in 1964 and ruled until 1985, after which civilian governance resumed. Brazil's current constitution, formulated in 1988, defines it as a democratic federal republic. Due to its rich culture and history, the country ranks thirteenth in the world by number of UNESCO World Heritage Sites.
"""

### Tokenize the text into sentences

In [4]:
tokens = nltk.sent_tokenize(input_text, language="english")
# tokens

### Sliding window functionality is one of the two important steps of the algorithm

Image below for visualization of the process.

In [5]:
def create_sliding_windows(tokens: list[str]) -> list[str]:
    i = 0
    embedding_tokens = []
    while i < len(tokens):
        if i >= len(tokens):
            break 

        k = i+2
        if k >= len(tokens):
            k = len(tokens)

        o = " ".join(tokens[i:k])
        embedding_tokens.append(o)
        
        i +=1
    
    return embedding_tokens

tokens_for_embedding = create_sliding_windows(tokens)
# tokens_for_embedding


### Generic implementation of calculating the embeddings of the tokens
The specifics of embeddings isn't important here. Follow best practices in order to chose the right embedding model for the intended application.

In [6]:
def calculate_embeddings(tokens: list[str]) -> list[np.ndarray]:
    promptStruct = {
    "model": "nomic-embed-text",
    "prompt": ""
    }

    embeddings = []
    for token in tokens:
        promptStruct["prompt"] = token
        resp = rq.post("http://localhost:11434/api/embeddings", data=json.dumps(promptStruct))
        if resp.status_code != 200:
            raise Exception("Embedding response failed")

        embeddings.append(np.array(resp.json()["embedding"]))
    
    return embeddings

token_embeddings = calculate_embeddings(tokens_for_embedding)

### Generic implementation of calculating the cosine similarity between the embeddings
The specifics of the cosine similarity is not critical either. Other similarity functions may be used as well, however has not been tested.

In [7]:
def calculate_similarities(embeddings) -> list[float]:
    similarities = []

    for i, e in enumerate(embeddings[:-1]):
        sim = pairwise.cosine_similarity([e], [embeddings[i+1]])
        similarities.append(sim[0][0])

    # Add two fake similarities to allow for the ending to be included
    # 1 are used to force a split to happen if a local minmum has not been
    # detected yet.
    similarities.append(1)
    similarities.append(1)

    return similarities

cos_similarities = calculate_similarities(token_embeddings)

In [8]:
# Only used for some visualization of the similarities vs tokens
# Prints out the similarities of the string and next string below it
# First similarity is the cosine similarity of the string at the same row, and the row below it
for i, _ in enumerate(tokens_for_embedding[:-1]):
    print("\t\t", tokens_for_embedding[i])
    print(cos_similarities[i])
    print("\t\t",tokens_for_embedding[i+1])
    print()

		 Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous.
0.9292469943847795
		 Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo.

		 Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo.
0.9103549635932817
		 Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District.

		 Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District.
0.867718675215865
		 Brazil is a federation composed of 26 states and a Federal District. It is the only country in the Americas where Portuguese is an official lang

### Building the new chunks

The goal here is to detect the local minimums and separate based on the local minimums. Then in order to know where the overlapping portion of the tokens go, the comparison is made between the previous and next similarity. The higher similarity combination get's the overlapping token as well.

In [9]:
# The delimiter is the delimiter placed between the different tokens merged together.
# In production it is recommended to use a single space as the delimiter
# For the purposes of the demonstration, a new line character is used instead.
DELIMITER = "\n"

In [10]:
def build_chunks(similarities: list[float], tokens: list[str]) -> list[str]:
    chunks = []
    chunk = f"{tokens[0]}"
    i = 1
    while i < len(similarities)-1:
        
        last_s = similarities[i-1]
        current_s = similarities[i]
        next_s = similarities[i+1]
        
        chunk = f"{chunk}{DELIMITER}{tokens[i]}"

        if last_s > current_s and next_s > current_s:

            # Key difference from overlapping approach, 
            # compare the previous vs the next similarities
            if last_s > next_s:
                chunk = f"{chunk}{DELIMITER}{tokens[i+1]}"
                chunks.append(chunk)
                i += 1 # Add the 1 to skip the next adding the next chunk
            else:
                chunks.append(chunk)
            

            chunk = f""
        
        i += 1
            
    chunks.append(chunk)

    return chunks

chunks = build_chunks(cos_similarities, tokens)

In [11]:
print_chunks_coloured(chunks)

Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and Latin America.
Brazil is the world's fifth-largest country by area and the seventh most populous.
Its capital is Brasília, and its most populous city is São Paulo.
Brazil is a federation composed of 26 states and a Federal District.

It is the only country in the Americas where Portuguese is an official language.
Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.

Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).
Covering roughly half of South America's land area, it borders all other countries and territories on the continent except Ecuador and Chile.

Brazil's Amazon basin includes a vast tropical forest home to diverse wildlife, a variety of ecological systems, and extensive natural resources spanning numerous protected habitats.
Th

### The algorithm can be used recursively to increase chunk size
If a larger chunk size is needed, the algorithm can be recursively run on itself.  
It is however important to remember that the purpose of chunking is to keep the semantic context similar within the chunk. The chunks are meant to be split in a way which allows for easy retreival to provide enhanced **context** further down the pipeline. Although the algorithm can run on itself, that may not always be necessary or desired.

In [12]:
recalculated_tokens_for_embedding = create_sliding_windows(chunks)

recalculated_token_embeddings = calculate_embeddings(recalculated_tokens_for_embedding)

recalculated_cos_similarities = calculate_similarities(recalculated_token_embeddings)

chunks = build_chunks(recalculated_cos_similarities, chunks)

print_chunks_coloured(chunks)

Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and Latin America.
Brazil is the world's fifth-largest country by area and the seventh most populous.
Its capital is Brasília, and its most populous city is São Paulo.
Brazil is a federation composed of 26 states and a Federal District.

It is the only country in the Americas where Portuguese is an official language.
Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.


Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).
Covering roughly half of South America's land area, it borders all other countries and territories on the continent except Ecuador and Chile.

Brazil's Amazon basin includes a vast tropical forest home to diverse wildlife, a variety of ecological systems, and extensive natural resources spanning numerous protected habitats.
T

#### EXTRA: A variation of the algorithm if overlap is needed
Overlap between chunks is often used to preserve cross-chunk context. Conceptually, overlap should not be necessary if the chunk and it's context is correctly calculated.  
The algorithm should not need an overlap due to it fundamentally being intended to chunk together relevant items, however if overlap is necessary - a variation of the algorithm
is provided below.

In [13]:
def build_chunks_with_overlapping(similarities: list[float], tokens: list[str]) -> list[str]:
    chunks = []
    chunk = f"{tokens[0]}"
    i = 1
    while i < len(similarities)-1:
        
        last_s = similarities[i-1]
        current_s = similarities[i]
        next_s = similarities[i+1]
        
        chunk = f"{chunk}{DELIMITER}{tokens[i]}"

        if last_s > current_s and next_s > current_s:
            chunk = f"{chunk}{DELIMITER}{tokens[i+1]}"
            chunks.append(chunk)

            chunk = f""
        
        i += 1
            
    chunks.append(chunk)

    return chunks

chunks_with_overlap = build_chunks_with_overlapping(cos_similarities, tokens)

In [14]:
print_chunks_coloured(chunks_with_overlap)

Brazil, officially the Federative Republic of Brazil, is the largest and easternmost country in South America and Latin America.
Brazil is the world's fifth-largest country by area and the seventh most populous.
Its capital is Brasília, and its most populous city is São Paulo.
Brazil is a federation composed of 26 states and a Federal District.

Brazil is a federation composed of 26 states and a Federal District.
It is the only country in the Americas where Portuguese is an official language.
Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.

Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.
Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).
Covering roughly half of South America's land area, it borders all other countries and territories on the continent

# More examples of texts

A piece from [Paul Graham's MIT essay](https://paulgraham.com/mit.html)
This essay is very "talkative" and is a good way of testing how the algorithm deals with chatty texts. The text has been put into one big block on purpose to make the algorithm have full control over finding the semantic chunks.

In [15]:
input_text = """Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be? I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups. I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's. The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college? In fact, why go to college at all? Why not start a startup instead? A year and a half ago I gave a talk where I said that the average age of the founders of Yahoo, Google, and Microsoft was 24, and that if grad students could start startups, why not undergrads? I'm glad I phrased that as a question, because now I can pretend it wasn't merely a rhetorical one. At the time I couldn't imagine why there should be any lower limit for the age of startup founders. Graduation is a bureaucratic change, not a biological one. And certainly there are undergrads as competent technically as most grad students. So why shouldn't undergrads be able to start startups as well as grad students? I now realize that something does change at graduation: you lose a huge excuse for failing. Regardless of how complex your life is, you'll find that everyone else, including your family and friends, will discard all the low bits and regard you as having a single occupation at any given time. If you're in college and have a summer job writing software, you still read as a student. Whereas if you graduate and get a job programming, you'll be instantly regarded by everyone as a programmer. The problem with starting a startup while you're still in school is that there's a built-in escape hatch. If you start a startup in the summer between your junior and senior year, it reads to everyone as a summer job. So if it goes nowhere, big deal; you return to school in the fall with all the other seniors; no one regards you as a failure, because your occupation is student, and you didn't fail at that. Whereas if you start a startup just one year later, after you graduate, as long as you're not accepted to grad school in the fall the startup reads to everyone as your occupation. You're now a startup founder, so you have to do well at that. For nearly everyone, the opinion of one's peers is the most powerful motivator of all—more powerful even than the nominal goal of most startup founders, getting rich. [1] About a month into each funding cycle we have an event called Prototype Day where each startup presents to the others what they've got so far. You might think they wouldn't need any more motivation. They're working on their cool new idea; they have funding for the immediate future; and they're playing a game with only two outcomes: wealth or failure. You'd think that would be motivation enough. And yet the prospect of a demo pushes most of them into a rush of activity.
"""

In [16]:
tokens = nltk.sent_tokenize(input_text, language="english")

tokens_for_embedding = create_sliding_windows(tokens)

token_embeddings = calculate_embeddings(tokens_for_embedding)

cos_similarities = calculate_similarities(token_embeddings)

chunks = build_chunks(cos_similarities, tokens)

print_chunks_coloured(chunks)

Till recently graduating seniors had two choices: get a job or go to grad school.
I think there will increasingly be a third option: to start your own startup.
But how common will that be?
I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school.
In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups.
I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's.

The most ambitious students will at this point be asking: Why wait till you graduate?
Why not start a startup while you're in college?
In fact, why go to college at all?
Why not start a startup instead?

A year and a half ago I gave a talk where I said that the average age of the founders of Yahoo, Google, and Microsoft was 24, and that if grad students could start startups, w

A piece from [Greg Kamradt tutorial](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb).
Pay close attention to how the chunker handles the "*You know? (like this sentenence)*". The text has been put into one big block on purpose to make the algorithm have full control over finding the semantic chunks.

In [17]:
input_text = """Isn't it weird that we have a global constant for chunk size? Isn't it even weirder that our normal chunking mechanisms don't take into account the actual content? I'm not the only one who thinks so. There has to be a better way - let's explore and find out. Embeddings represent the semantic meaning of a string. They don't do much on their own, but when compared to embeddings of other texts you can start to infer the relationship between chunks. I want to lean into this property and explore using embeddings to find clusters of semantically similar texts. The hypothesis is that semantically similar chunks should be held together. I tried a few methods: Heirarchical clustering with positional reward - I wanted to see how heirarchical clustering of sentence embeddings would do. But because I chose to split on sentences, there was an issue with small short sentences after a long one. You know? (like this last sentence). They could change the meaning of a chunk, so I added a positional reward and clusters were more likely to form if they were sentences next to each other. This ended up being ok, but tuning the parameters was slow and unoptimal. Find break points between sequential sentences - Next up I tried a walk method. I started at the first sentence, got the embedding, then compared it to sentence #2, then compared #2 and #3 and so on. I was looking for "break points" where embedding distance was large. If it was above a threshold, then I considered it the start of a new semantic section. I originally tried taking embeddings of every sentence, but this turned out to be too noisy. So I ended up taking groups of 3 sentences (a window), then got an embedding, then dropped the first sentence, and added the next one. This worked out a bit better. I'll show method #2 here - It's not perfect by any means, but it's a good starting point for an exploration and I'd love to hear about how you think it could be improved. First, let's load up our essay that we'll run through. I'm just doing a single essay here to keep the tokens down.
"""


In [18]:
tokens = nltk.sent_tokenize(input_text, language="english")

tokens_for_embedding = create_sliding_windows(tokens)

token_embeddings = calculate_embeddings(tokens_for_embedding)

cos_similarities = calculate_similarities(token_embeddings)

chunks = build_chunks(cos_similarities, tokens)

print_chunks_coloured(chunks)

Isn't it weird that we have a global constant for chunk size?
Isn't it even weirder that our normal chunking mechanisms don't take into account the actual content?
I'm not the only one who thinks so.

There has to be a better way - let's explore and find out.
Embeddings represent the semantic meaning of a string.

They don't do much on their own, but when compared to embeddings of other texts you can start to infer the relationship between chunks.
I want to lean into this property and explore using embeddings to find clusters of semantically similar texts.
The hypothesis is that semantically similar chunks should be held together.

I tried a few methods: Heirarchical clustering with positional reward - I wanted to see how heirarchical clustering of sentence embeddings would do.
But because I chose to split on sentences, there was an issue with small short sentences after a long one.
You know?

(like this last sentence).
They could change the meaning of a chunk, so I added a positional 